# MISS meets TRAK

MISS for non-linear model: linearizing as TRAK

> The target function is assumed to be the raw logit, i.e., $\phi(x_{\text{test}}) = \theta^\top x_{\text{test}} + b$.

In [2]:
from model_train import MLP, data_generation
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Sampler
from pydvl.influence.torch import EkfacInfluence

Using device: cpu


/home/pbb/miniconda3/envs/influence/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
MISS = torch.zeros(1, 20, dtype=torch.int32)
index = list(range(5000))
step_size = 5
max_idx_list = [2, 5, 3, 2, 1]
for i in range(0, 20, 5):
    print(i)
    # handle overflow
    if i + step_size >= 20:
        step_size = 20 - i
        max_idx_list = max_idx_list[:step_size]
    # print(MISS[0, i:i+step_size])
    MISS[0, i:i+step_size] = torch.tensor([index[l] for l in max_idx_list])
    index = [index[i] for i in range(len(index)) if i not in max_idx_list]
print(MISS)

0
5
10
15
!
tensor([[ 2,  5,  3,  2,  1,  6,  9,  7,  6,  4, 10, 13, 11, 10,  8, 14, 17, 15,
         14, 12]], dtype=torch.int32)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SubsetSamper(Sampler):
    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return iter(self.indices)

    def __len__(self):
        return len(self.indices)

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1, sampler=SubsetSamper(list(range(500))))
test_loader = DataLoader(test_dataset, batch_size=1, sampler=SubsetSamper(list(range(50))))

influence_model = EkfacInfluence(
                MLP().to(device),
                update_diagonal=True,
                hessian_regularization=0.001,
            )
influence_model = influence_model.fit(train_loader)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [3]:
checkpoint_files =  [f"./checkpoint/seed_{0}_ensemble_{i}.pt" for i in range(1)]

train_loader, test_loader = data_generation(list(range(500)), list(range(10)), mode='MISS')

In [4]:
# IF
IF = MISS_IF(model=MLP().to("cpu"),
                model_checkpoints=checkpoint_files,
                train_loader=train_loader,
                test_loader=test_loader,
                model_output_class=MNISTModelOutput,
                device=device)

# Retrain the model without the most influential samples for every test point
MISS = IF.adaptive_most_k(3)

100%|██████████| 1/1 [00:02<00:00,  2.47s/it]


Start TRAK greedy


100%|██████████| 10/10 [00:00<00:00, 20410.24it/s]


tensor([[7],
        [6],
        [0],
        [6],
        [0],
        [9],
        [8],
        [6],
        [1],
        [8]], dtype=torch.int32)



100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


Start TRAK greedy


  0%|          | 0/10 [00:09<?, ?it/s]


KeyboardInterrupt: 